# Tutorial : TermSheetTransformer

Ce notebook présente comment utiliser **TermSheetTransformer** pour modifier des documents Word de type term sheets et générer automatiquement trois versions :

1. **Mark-up** : document avec les modifications visibles (jaune + barré)
2. **Final** : document modifié sans marquage
3. **PDF** : version PDF du document final

## 1. Installation et import

In [ ]:
# Installation des dépendances (à exécuter une fois)
# !pip install python-docx docx2pdf

from ts_editor_D import TermSheetTransformer
from pathlib import Path

## 2. Initialisation

On crée un transformer en passant le chemin vers le document Word. **Les fichiers .doc sont automatiquement convertis en .docx** via LibreOffice (doit être installé).

In [ ]:
# Chemin vers votre document Word (.docx ou .doc)
CHEMIN_DOCUMENT = "mon_term_sheet.docx"  # ou "mon_term_sheet.doc" → conversion auto

transformer = TermSheetTransformer(CHEMIN_DOCUMENT)

## 3. Déclaration des modifications

L'API est **fluide** : chaque méthode retourne `self`, ce qui permet d'enchaîner les appels.

### 3.1 Remplacement de mots

Remplace toutes les occurrences d'un mot par un autre dans tout le document.

In [ ]:
transformer.replace_word("Mot1", "Mot2")
transformer.replace_word("Ancienne valeur", "Nouvelle valeur")

### 3.2 Ajout d'une section

Insère une nouvelle ligne (section) après une section existante, en conservant le style.

In [ ]:
transformer.add_section_after(
    after_section="Listing",      # Titre de la section après laquelle insérer
    title="Issuer",                # Titre de la nouvelle section
    description="Paul Berber"      # Description
)

### 3.3 Modification d'une description

Modifie la description d'une section existante (cellule de droite).

In [ ]:
transformer.update_section_description(
    section_title="Country",
    new_description="France"
)

### 3.4 Suppression d'une section

In [ ]:
transformer.remove_section("Old Section")

### 3.5 Ajout du logo

Supprime le header existant et place uniquement le logo en haut à gauche de chaque page.

In [ ]:
transformer.add_logo(
    logo_path="logo_nbc.png",
    width_inches=0.8
)

### 3.6 Paragraphes hors tableau (texte libre)

Pour les parties du document qui ne sont pas dans des tableaux :

In [ ]:
# Ajouter un titre/description (optionnel : surligné rouge dans la version finale)
transformer.add_content("Notice importante", red_highlight_in_final=True)
transformer.add_content("Paragraphe à la fin du document")
transformer.add_content("Paragraphe après un autre", after_paragraph="Texte existant")

# Modifier un paragraphe existant
transformer.update_paragraph("Ancien texte", "Nouveau texte")

# Supprimer un paragraphe
transformer.remove_paragraph("Paragraphe à supprimer")

## 4. Exécution et export

Une fois toutes les modifications déclarées, on lance l'exécution. Les trois fichiers sont générés dans le dossier de sortie.

In [ ]:
transformer.execute_and_export(
    output_dir="./output",
    markup_docx="modifications.docx",   # Version avec modifications visibles
    final_docx="document_final.docx",  # Version propre
    final_pdf="document_final.pdf"     # PDF (nécessite Word installé)
)

## 5. Règles de marquage

**Version Mark-up** : changements visibles (jaune + barré). **Version finale** : aucun surlignage (sauf rouge si `red_highlight_in_final=True`).

| Type | Rendu Mark-up |
|------|---------------|
| Remplacement (Mot1 → Mot2) | Mot1 barré + Mot2 jaune |
| Ajout | Contenu jaune |
| Modification | Nouveau texte jaune |
| Suppression | Texte barré |

## 6. Exemple complet en une seule fois

In [ ]:
# Réinitialiser et tout faire en chaîne
transformer = (
    TermSheetTransformer(CHEMIN_DOCUMENT)
    .replace_word("Mot1", "Mot2")
    .add_section_after("Listing", "Issuer", "Paul Berber")
    .update_section_description("Country", "France")
    .remove_section("Section obsolète")
    .add_content("Avertissement", red_highlight_in_final=True)
    .update_paragraph("Texte à remplacer", "Nouveau contenu")
    .remove_paragraph("Paragraphe obsolète")
    .add_logo("logo_nbc.png", width_inches=0.8)
)

transformer.execute_and_export(
    output_dir="./output",
    markup_docx="modifications.docx",
    final_docx="document_final.docx",
    final_pdf="document_final.pdf"
)

## 7. Paramètre `occurrence`

Si une section apparaît plusieurs fois dans le document (même titre), on peut cibler la n-ième occurrence :

In [ ]:
# Cibler la 2ème section "Listing"
transformer.add_section_after("Listing", "Nouvelle section", "Description", occurrence=2)

# Modifier la 1ère section "Country" (par défaut)
transformer.update_section_description("Country", "France", occurrence=1)

## 8. Gestion des erreurs

Si une section est introuvable, une `ValueError` est levée :

In [ ]:
try:
    transformer = TermSheetTransformer(CHEMIN_DOCUMENT)
    transformer.update_section_description("Section inexistante", "Nouvelle valeur")
    transformer.execute_and_export("./output")
except ValueError as e:
    print(f"Erreur : {e}")

---

**Note** : La conversion PDF nécessite Microsoft Word installé sur la machine (via `docx2pdf`). Si la conversion échoue, les fichiers Word sont tout de même générés.